In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("divorce.csv", low_memory=False)
df.head()

,Atr1,Atr2,Atr3,Atr4,Atr5,Atr6,Atr7,Atr8,Atr9,Atr10,...,Atr46,Atr47,Atr48,Atr49,Atr50,Atr51,Atr52,Atr53,Atr54,Class
0,2,2,4,1,0,0,0,0,0,0,...,2,1,3,3,3,2,3,2,1,1
1,4,4,4,4,4,0,0,4,4,4,...,2,2,3,4,4,4,4,2,2,1
2,2,2,2,2,1,3,2,1,1,2,...,3,2,3,1,1,1,2,2,2,1
3,3,2,3,2,3,3,3,3,3,3,...,2,2,3,3,3,3,2,2,2,1
4,2,2,1,1,1,1,0,0,0,0,...,2,1,2,3,2,2,2,1,0,1


## Differences in Means (Divorced vs. Married)

In [3]:
# Create masks for divorced and married respondents
divorced = df["Class"] == 1 # 1 means divorced
married = df["Class"] == 0 # 0 means stayed divorced

# 0 means agree, 4 means disagree
print(df[divorced]["Atr1"].mean())
print(df[married]["Atr1"].mean())

3.1904761904761907
0.3953488372093023


In [4]:
def divorced_vs_not(question_num):
    """
    Calculates the average response of divorced respondents minus the
    average response of married respondents to a given question
    
    question_num: the integer number of the question
    """
    column_name = "Atr" + str(question_num)
    divorced_avg = df[divorced][column_name].mean()
    married_avg = df[married][column_name].mean()
    return divorced_avg - married_avg

In [5]:
# Find the most 10 questions with the highest differences
pairs = [(i, divorced_vs_not(i)) for i in range(1, 55)]
pairs.sort(key=lambda x: x[1], reverse=True)

for pair in pairs[:10]:
    print(pair)

(40, 3.3621262458471763)
(36, 3.179401993355482)
(35, 3.169158361018826)
(39, 3.0730897009966776)
(41, 3.070874861572536)
(33, 3.0664451827242525)
(38, 3.055924695459579)
(19, 3.0390365448504983)
(11, 3.0166112956810633)
(17, 2.992248062015504)


## Predicting: Neural Network

In [6]:
df.shape

(170, 55)

In [7]:
# Normalize data
for i in range(1, 55):
    df["Atr"+str(i)] /= 4
df.head()

,Atr1,Atr2,Atr3,Atr4,Atr5,Atr6,Atr7,Atr8,Atr9,Atr10,...,Atr46,Atr47,Atr48,Atr49,Atr50,Atr51,Atr52,Atr53,Atr54,Class
0,0.50,0.5,1.00,0.25,0.00,0.00,0.00,0.00,0.00,0.00,...,0.50,0.25,0.75,0.75,0.75,0.50,0.75,0.50,0.25,1
1,1.00,1.0,1.00,1.00,1.00,0.00,0.00,1.00,1.00,1.00,...,0.50,0.50,0.75,1.00,1.00,1.00,1.00,0.50,0.50,1
2,0.50,0.5,0.50,0.50,0.25,0.75,0.50,0.25,0.25,0.50,...,0.75,0.50,0.75,0.25,0.25,0.25,0.50,0.50,0.50,1
3,0.75,0.5,0.75,0.50,0.75,0.75,0.75,0.75,0.75,0.75,...,0.50,0.50,0.75,0.75,0.75,0.75,0.50,0.50,0.50,1
4,0.50,0.5,0.25,0.25,0.25,0.25,0.00,0.00,0.00,0.00,...,0.50,0.25,0.50,0.75,0.50,0.50,0.50,0.25,0.00,1


In [8]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

# Get training data
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]

# Slice attributes and labels
train_data = train.iloc[:,:54]
train_labels = train.iloc[:,54:]

test_data = test.iloc[:,:54]
test_labels = test.iloc[:,54:]

# Build the model.
model = Sequential()
model.add(Dense(20, activation='relu', input_dim=54))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model, iterating on the data in batches of 32 samples
model.fit(train_data, train_labels, epochs=1000, batch_size=32)

Using TensorFlow backend.


Epoch 1/1000
144/144 [==============================] - 0s 757us/step - loss: 0.6813 - accuracy: 0.5278
Epoch 2/1000
144/144 [==============================] - 0s 41us/step - loss: 0.6345 - accuracy: 0.8750
Epoch 3/1000
144/144 [==============================] - 0s 61us/step - loss: 0.5956 - accuracy: 0.9653
Epoch 4/1000
144/144 [==============================] - 0s 47us/step - loss: 0.5618 - accuracy: 0.9653
Epoch 5/1000
144/144 [==============================] - 0s 61us/step - loss: 0.5279 - accuracy: 0.9792
Epoch 6/1000
144/144 [==============================] - 0s 43us/step - loss: 0.4952 - accuracy: 0.9722
Epoch 7/1000
144/144 [==============================] - 0s 78us/step - loss: 0.4628 - accuracy: 0.9792
Epoch 8/1000
144/144 [==============================] - 0s 48us/step - loss: 0.4308 - accuracy: 0.9931
Epoch 9/1000
144/144 [==============================] - 0s 47us/step - loss: 0.3926 - accuracy: 0.9931
Epoch 10/1000
144/144 [==============================] - 0s 46us/step - 

144/144 [==============================] - 0s 39us/step - loss: 7.6744e-04 - accuracy: 1.0000
Epoch 80/1000
144/144 [==============================] - 0s 38us/step - loss: 6.8939e-04 - accuracy: 1.0000
Epoch 81/1000
144/144 [==============================] - 0s 33us/step - loss: 6.1063e-04 - accuracy: 1.0000
Epoch 82/1000
144/144 [==============================] - 0s 36us/step - loss: 5.6141e-04 - accuracy: 1.0000
Epoch 83/1000
144/144 [==============================] - 0s 35us/step - loss: 5.1683e-04 - accuracy: 1.0000
Epoch 84/1000
144/144 [==============================] - 0s 39us/step - loss: 4.6505e-04 - accuracy: 1.0000
Epoch 85/1000
144/144 [==============================] - 0s 32us/step - loss: 4.0384e-04 - accuracy: 1.0000
Epoch 86/1000
144/144 [==============================] - 0s 31us/step - loss: 3.6181e-04 - accuracy: 1.0000
Epoch 87/1000
144/144 [==============================] - 0s 37us/step - loss: 3.3033e-04 - accuracy: 1.0000
Epoch 88/1000
144/144 [===================

144/144 [==============================] - 0s 37us/step - loss: 1.0199e-07 - accuracy: 1.0000
Epoch 155/1000
144/144 [==============================] - 0s 35us/step - loss: 9.5160e-08 - accuracy: 1.0000
Epoch 156/1000
144/144 [==============================] - 0s 32us/step - loss: 8.5661e-08 - accuracy: 1.0000
Epoch 157/1000
144/144 [==============================] - 0s 37us/step - loss: 7.5173e-08 - accuracy: 1.0000
Epoch 158/1000
144/144 [==============================] - 0s 31us/step - loss: 6.7603e-08 - accuracy: 1.0000
Epoch 159/1000
144/144 [==============================] - 0s 34us/step - loss: 6.9083e-08 - accuracy: 1.0000
Epoch 160/1000
144/144 [==============================] - 0s 32us/step - loss: 5.9046e-08 - accuracy: 1.0000
Epoch 161/1000
144/144 [==============================] - 0s 31us/step - loss: 5.5679e-08 - accuracy: 1.0000
Epoch 162/1000
144/144 [==============================] - 0s 37us/step - loss: 5.1402e-08 - accuracy: 1.0000
Epoch 163/1000
144/144 [==========

144/144 [==============================] - 0s 30us/step - loss: 4.6616e-09 - accuracy: 1.0000
Epoch 230/1000
144/144 [==============================] - 0s 38us/step - loss: 4.5576e-09 - accuracy: 1.0000
Epoch 231/1000
144/144 [==============================] - 0s 31us/step - loss: 4.4697e-09 - accuracy: 1.0000
Epoch 232/1000
144/144 [==============================] - 0s 29us/step - loss: 4.3944e-09 - accuracy: 1.0000
Epoch 233/1000
144/144 [==============================] - 0s 39us/step - loss: 4.3537e-09 - accuracy: 1.0000
Epoch 234/1000
144/144 [==============================] - 0s 30us/step - loss: 4.2565e-09 - accuracy: 1.0000
Epoch 235/1000
144/144 [==============================] - 0s 33us/step - loss: 4.1783e-09 - accuracy: 1.0000
Epoch 236/1000
144/144 [==============================] - 0s 32us/step - loss: 4.1463e-09 - accuracy: 1.0000
Epoch 237/1000
144/144 [==============================] - 0s 32us/step - loss: 4.0687e-09 - accuracy: 1.0000
Epoch 238/1000
144/144 [==========

...

144/144 [==============================] - 0s 38us/step - loss: 6.2338e-10 - accuracy: 1.0000
Epoch 680/1000
144/144 [==============================] - 0s 33us/step - loss: 6.2420e-10 - accuracy: 1.0000
Epoch 681/1000
144/144 [==============================] - 0s 32us/step - loss: 6.2537e-10 - accuracy: 1.0000
Epoch 682/1000
144/144 [==============================] - 0s 34us/step - loss: 6.2808e-10 - accuracy: 1.0000
Epoch 683/1000
144/144 [==============================] - 0s 31us/step - loss: 6.2030e-10 - accuracy: 1.0000
Epoch 684/1000
144/144 [==============================] - 0s 32us/step - loss: 6.2101e-10 - accuracy: 1.0000
Epoch 685/1000
144/144 [==============================] - 0s 36us/step - loss: 6.2428e-10 - accuracy: 1.0000
Epoch 686/1000
144/144 [==============================] - 0s 30us/step - loss: 6.2420e-10 - accuracy: 1.0000
Epoch 687/1000
144/144 [==============================] - 0s 35us/step - loss: 6.2753e-10 - accuracy: 1.0000
Epoch 688/1000
144/144 [==========

144/144 [==============================] - 0s 37us/step - loss: 5.9024e-10 - accuracy: 1.0000
Epoch 755/1000
144/144 [==============================] - 0s 29us/step - loss: 5.9173e-10 - accuracy: 1.0000
Epoch 756/1000
144/144 [==============================] - 0s 35us/step - loss: 5.9392e-10 - accuracy: 1.0000
Epoch 757/1000
144/144 [==============================] - 0s 32us/step - loss: 5.9462e-10 - accuracy: 1.0000
Epoch 758/1000
144/144 [==============================] - 0s 35us/step - loss: 5.9703e-10 - accuracy: 1.0000
Epoch 759/1000
144/144 [==============================] - 0s 31us/step - loss: 5.8696e-10 - accuracy: 1.0000
Epoch 760/1000
144/144 [==============================] - 0s 34us/step - loss: 5.8987e-10 - accuracy: 1.0000
Epoch 761/1000
144/144 [==============================] - 0s 30us/step - loss: 5.9012e-10 - accuracy: 1.0000
Epoch 762/1000
144/144 [==============================] - 0s 29us/step - loss: 5.9118e-10 - accuracy: 1.0000
Epoch 763/1000
144/144 [==========

144/144 [==============================] - 0s 34us/step - loss: 5.7153e-10 - accuracy: 1.0000
Epoch 830/1000
144/144 [==============================] - 0s 31us/step - loss: 5.7352e-10 - accuracy: 1.0000
Epoch 831/1000
144/144 [==============================] - 0s 41us/step - loss: 5.7483e-10 - accuracy: 1.0000
Epoch 832/1000
144/144 [==============================] - 0s 31us/step - loss: 5.6503e-10 - accuracy: 1.0000
Epoch 833/1000
144/144 [==============================] - 0s 37us/step - loss: 5.6568e-10 - accuracy: 1.0000
Epoch 834/1000
144/144 [==============================] - 0s 34us/step - loss: 5.6808e-10 - accuracy: 1.0000
Epoch 835/1000
144/144 [==============================] - 0s 35us/step - loss: 5.6878e-10 - accuracy: 1.0000
Epoch 836/1000
144/144 [==============================] - 0s 35us/step - loss: 5.7025e-10 - accuracy: 1.0000
Epoch 837/1000
144/144 [==============================] - 0s 29us/step - loss: 5.7199e-10 - accuracy: 1.0000
Epoch 838/1000
144/144 [==========

144/144 [==============================] - 0s 31us/step - loss: 5.5565e-10 - accuracy: 1.0000
Epoch 905/1000
144/144 [==============================] - 0s 38us/step - loss: 5.5857e-10 - accuracy: 1.0000
Epoch 906/1000
144/144 [==============================] - 0s 38us/step - loss: 5.4773e-10 - accuracy: 1.0000
Epoch 907/1000
144/144 [==============================] - 0s 30us/step - loss: 5.4829e-10 - accuracy: 1.0000
Epoch 908/1000
144/144 [==============================] - 0s 44us/step - loss: 5.5000e-10 - accuracy: 1.0000
Epoch 909/1000
144/144 [==============================] - 0s 34us/step - loss: 5.5068e-10 - accuracy: 1.0000
Epoch 910/1000
144/144 [==============================] - 0s 33us/step - loss: 5.5222e-10 - accuracy: 1.0000
Epoch 911/1000
144/144 [==============================] - 0s 33us/step - loss: 5.5374e-10 - accuracy: 1.0000
Epoch 912/1000
144/144 [==============================] - 0s 33us/step - loss: 5.5500e-10 - accuracy: 1.0000
Epoch 913/1000
144/144 [==========

144/144 [==============================] - 0s 40us/step - loss: 5.3879e-10 - accuracy: 1.0000
Epoch 980/1000
144/144 [==============================] - 0s 35us/step - loss: 5.3980e-10 - accuracy: 1.0000
Epoch 981/1000
144/144 [==============================] - 0s 32us/step - loss: 5.4133e-10 - accuracy: 1.0000
Epoch 982/1000
144/144 [==============================] - 0s 40us/step - loss: 5.4273e-10 - accuracy: 1.0000
Epoch 983/1000
144/144 [==============================] - 0s 31us/step - loss: 5.3396e-10 - accuracy: 1.0000
Epoch 984/1000
144/144 [==============================] - 0s 35us/step - loss: 5.3541e-10 - accuracy: 1.0000
Epoch 985/1000
144/144 [==============================] - 0s 29us/step - loss: 5.3679e-10 - accuracy: 1.0000
Epoch 986/1000
144/144 [==============================] - 0s 35us/step - loss: 5.3741e-10 - accuracy: 1.0000
Epoch 987/1000
144/144 [==============================] - 0s 38us/step - loss: 5.3898e-10 - accuracy: 1.0000
Epoch 988/1000
144/144 [==========

In [9]:
model.evaluate(test_data, test_labels, batch_size=32)

26/26 [==============================] - 0s 705us/step


[1.4261956214904785, 0.9230769276618958]

# Categorizing Questions

In [10]:
categories = pd.read_csv("divorce_values.csv")
categories.tail()

,Atr1,Atr2,Atr3,Atr4,Atr5,Atr6,Atr7,Atr8,Atr9,Atr10,...,Atr46,Atr47,Atr48,Atr49,Atr50,Atr51,Atr52,Atr53,Atr54,Unnamed: 54
0,Communications,Communications,Communications,Communications,Experiences,Experiences,Experiences,Experiences,Experiences,Values,...,Acquiesce,Acquiesce,Self-righteous,Self-righteous,Self-righteous,Self-righteous,Respect,Respect,Respect,Zero
1,Positive,Positive,Positive,Positive,Positive,Negative,Negative,Positive,Positive,Positive,...,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Zero


In [11]:
for i in range(1, 55):
    #print(divorce_categories["Atr" + str(i)].shape)
    if categories["Atr" + str(i)].iloc[1] == 'Negative':
        #print(4 - divorce_categories["Atr" + str(i)])
        df["Atr" + str(i)] = 1 - df["Atr" + str(i)]

In [12]:
unique_categories = categories.iloc[0,:54].unique()
category_avgs = []
for row in range(len(df)):
    avgs = []
    for category in unique_categories:
        values = []
        for col in range(54):
            if categories.iloc[0, col] == category:
                values.append(df.iloc[row,col])
        avgs.append(sum(values) / len(values))
    avgs.append(df.iloc[row,54])
    category_avgs.append(avgs)

headers = list(unique_categories)
headers.append("Divorce Status")
avg_df = pd.DataFrame(category_avgs, columns=headers)
export_csv = avg_df.to_csv(r'./output.csv', index=None, header=True)
avg_df.head()

,Communications,Experiences,Values,Empathy,Respect,Acquiesce,Self-righteous,Divorce Status
0,0.50000,0.321429,0.100,0.027778,0.625,0.583333,0.3125,1
1,0.62500,0.964286,0.750,0.277778,0.450,0.375000,0.0625,1
2,0.43750,0.464286,0.625,0.444444,0.550,0.375000,0.6250,1
3,0.40625,0.642857,0.750,0.333333,0.500,0.416667,0.2500,1
4,0.59375,0.321429,0.225,0.194444,0.825,0.583333,0.4375,1


In [13]:
divorced_mask = avg_df["Divorce Status"] == 1 # 1 means divorced
married_mask = avg_df["Divorce Status"] == 0 # 0 means stayed divorced

for category in unique_categories:
    divorced_avg = avg_df[divorced_mask][category].mean()
    married_avg = avg_df[married_mask][category].mean()
    print(category)
    print(abs(divorced_avg - married_avg), divorced_avg, married_avg)

Communications
0.08209613787375408 0.42299107142857145 0.5050872093023255
Experiences
0.4374999999999999 0.7410714285714285 0.3035714285714286
Values
0.6760866555924697 0.7342261904761906 0.05813953488372092
Empathy
0.6468100159960625 0.6878306878306878 0.041020671834625315
Respect
0.6777547065337763 0.15654761904761902 0.8343023255813954
Acquiesce
0.48397702104097473 0.16815476190476195 0.6521317829457367
Self-righteous
0.46849044850498334 0.13616071428571427 0.6046511627906976
